# 🧪 Finance ABAC Demo - Step 3: Setup Governed Tags

## 📋 Overview
This notebook creates the tag policies defining the necessary governed tags and their allowed values

### What This Notebook Does:
1. **Creates Tag policy**: Uses REST API to create tag policies defining governed tags and their allowed values
2. **Demonstrates Usage**: Shows how to apply tags to relevant columns in the finance datasets

## 🎓 How to Use This Notebook
1. **Ensure Steps 1-2 Complete**: All functions, tables, and data must exist
2. **Run All Cells**: Execute sequentially to see all test results
3. **Verify Expectations**: Check that tags have been applied through the catalog explorer

## ⚙️ Prerequisites
- ✅ **Step 1 completed**: All functions created
- ✅ **Step 2 completed**: Core tables with data
- ✅ APPLY TAG permission on all tables

## 📊 Expected Results
After running this notebook:
- ✅ Tag policies are created
- ✅ Governed tags assigned
- ✅ Prepared for creating ABAC policies
---


In [0]:
pip install pyyaml

In [0]:
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path
import requests
import os
from databricks.sdk import WorkspaceClient

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'finance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use for the cells below
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")

client = WorkspaceClient()
workspace_url = client.config.host

In [0]:
%sql
SELECT '🎯 Target: ' || current_catalog() || '.' || current_schema() AS status;

###Define Governed Tags + Allowed Values
https://docs.databricks.com/api/workspace/tagpolicies/createtagpolicy

In [0]:
def get_token():
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return getattr(ctx, "apiToken")().get()


def create_tag_policy(payload):
    data = requests.post(
        f"{workspace_url}/api/2.1/tag-policies",
        headers={"Authorization": f"Bearer {get_token()}"},
        json=payload,
    )

    return data

In [0]:
pii_payload = {
    "description": "PII field types",
    "tag_key": "pii_type",
    "values": [
        {"name": "ssn"},
        {"name": "email"},
        {"name": "location"},
        {"name": "phone"},
        {"name": "income"},
        {"name": "account"},
        {"name": "routing_number"},
        {"name": "ip_address"},
        {"name": "credit_card"},
        {"name": "transaction_amount"},
        {"name": "transaction_id"},
        {"name": "id"}
    ]
}
print(create_tag_policy(pii_payload).json())


pci_payload = {
    "description": "PCI-DSS compliance requirement",
    "tag_key": "pci_compliance",
    "values": [
        {"name": "Required"},
        {"name": "Not_Required"}
    ]
}
print(create_tag_policy(pci_payload).json())


classification_payload = {
    "description": "Data classification level",
    "tag_key": "data_classification",
    "values": [
        {"name": "Confidential"},
        {"name": "Internal"},
        {"name": "Public"}
    ]
}
print(create_tag_policy(classification_payload).json())


fraud_payload = {
    "description": "Identify fraud detection flag",
    "tag_key": "fraud_detection",
    "values": [
        {"name": "true"},
        {"name": "false"}
    ]
}
print(create_tag_policy(fraud_payload).json())

In [0]:
%sql
-- =============================================
-- TAGS FOR: customers (PII data)
-- =============================================
ALTER TABLE customers ALTER COLUMN ssn SET TAGS ('pii_type' = 'ssn', 'pci_compliance' = 'Required');
ALTER TABLE customers ALTER COLUMN email SET TAGS ('pii_type' = 'email');
ALTER TABLE customers ALTER COLUMN phone SET TAGS ('pii_type' = 'phone');
ALTER TABLE customers ALTER COLUMN state SET TAGS ('pii_type' = 'location');
ALTER TABLE customers ALTER COLUMN annual_income SET TAGS ('pii_type' = 'income', 'data_classification' = 'Confidential');
ALTER TABLE customers ALTER COLUMN customer_id SET TAGS ('pii_type' = 'id');

In [0]:
%sql
-- =============================================
-- TAGS FOR: accounts (Account numbers)
-- =============================================
ALTER TABLE accounts ALTER COLUMN account_number SET TAGS ('pii_type' = 'account', 'data_classification' = 'Confidential');
ALTER TABLE accounts ALTER COLUMN routing_number SET TAGS ('pii_type' = 'routing_number', 'data_classification' = 'Confidential');
ALTER TABLE accounts ALTER COLUMN customer_id SET TAGS ('pii_type' = 'id');

In [0]:
%sql
-- =============================================
-- TAGS FOR: credit_cards (PCI-DSS data)
-- =============================================
ALTER TABLE credit_cards ALTER COLUMN card_number SET TAGS ('pii_type' = 'credit_card', 'pci_compliance' = 'Required');
ALTER TABLE credit_cards ALTER COLUMN customer_id SET TAGS ('pii_type' = 'id');

In [0]:
%sql
-- =============================================
-- TAGS FOR: transactions (Transaction data)
-- =============================================
ALTER TABLE transactions ALTER COLUMN ip_address SET TAGS ('pii_type' = 'ip_address');
ALTER TABLE transactions ALTER COLUMN amount SET TAGS ('pii_type' = 'transaction_amount');
ALTER TABLE transactions ALTER COLUMN customer_id SET TAGS ('pii_type' = 'id');
ALTER TABLE transactions ALTER COLUMN transaction_id SET TAGS ('pii_type' = 'transaction_id');
ALTER TABLE transactions ALTER COLUMN fraud_flag SET TAGS ('fraud_detection' = 'true');

In [0]:
%sql
SELECT '✅ Tags applied successfully to finance tables!' AS status;

## ✅ Success!

Tag policies have been created successfully and governed tags have been assigned!

### What You Just Created:
- ✅ Governed tags for capturing data sensitivity 
- ✅ Tag assignment to tables

### 🎯 Next Step:

Continue to **`4_Test_ABAC_Policies.ipynb`** to define ABAC policies using governed tags and test them on each dataset in the finance schema

---